### Trial & Error
- break: target_date 기준으로 break해버려서 모든 키워드 검색이 이루어지지 않음. 키워드 검색 반복문을 중간에 break하기 때문에 첫 번째 키워드만 검색됨

## 부정적 뉴스 모니터링 자동화
- 개발목적: 혐의자가 특정되는 금융 부정적 뉴스 모니터링 자동화
- 기대효과: 데일리 모니터링 시간 단축 및 정확성 향상

- 익명
  ```
  ANON_PATTERNS = [
      r"\b[A-Z]씨\b",                 # A씨
      r"\b[가-힣]씨\b",                # 김씨(성 1글자 + 씨)
      r"\b[가-힣]모\s?씨\b",           # 김모씨
      r"[가-힣]○{1,}",                 # 김○○
      r"[가-힣]ㅇ{1,}",                 # 김ㅇㅇ
      r"[가-힣]\*{1,}",                 # 김**
      r"\b\d{2,}대\s?(남성|여성)\b",   # 40대 남성
      r"\b피의자\b|\b혐의자\b"         # 실명 대신 역할만
  ]
  ```


In [2]:
# !pip install konlpy
!pip install python-mecab-ko

In [26]:
import pandas as pd
import urllib.request
import urllib.parse
import json
import re, html, time
from datetime import datetime
from google.colab import userdata
import re
import pandas as pd
from mecab import MeCab
mecab = MeCab()

# from konlpy.tag import Komoran
# komoran = Komoran()
# from konlpy.tag import Okt
# okt = Okt()

### 1. 뉴스 크롤링
- 네이버뉴스 API 이용
- 조회기간: target_date ~ 오늘(현재)날짜 (target_date: 입력값)
- "자금세탁", "배임", "횡령", "탈세" 키워드 동시 조회

In [4]:
# 1. 뉴스 크롤링

# API 인증
client_id = userdata.get('NAVER_CLIENT_ID')
client_secret = userdata.get('NAVER_CLIENT_SECRET')

if not client_id or not client_secret:
    raise ValueError("NAVER_CLIENT_ID / NAVER_CLIENT_SECRET가 설정되지 않았습니다.")

# 파라미터 선언
base_url = "https://openapi.naver.com/v1/search/news.json"
page_num = 100
sort = "date"
max_start = 1000
max_calls = 2000

# 텍스트 데이터 전처리
def clean_html(text: str) -> str:
    text = html.unescape(text)
    text = re.sub(r"<[^>]+>", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# 뉴스기사 날짜 전처리
def parse_pubdate(pubdate_str: str) -> datetime:
    # 예: Mon, 26 Sep 2016 07:50:00 +0900
    return datetime.strptime(pubdate_str, "%a, %d %b %Y %H:%M:%S +0900")

# 크롤링- target_date_str보다 과거 날짜가 나오기 시작하면 break
def get_raw_and_clean_df(keywords, target_date_str, sleep_sec=0.1):

    raw_rows = []
    api_calls = 0

    for kw in keywords:
        enc_kw = urllib.parse.quote(kw)

        for start in range(1, max_start + 1, page_num):
            if api_calls >= max_calls:
                print("[WARN] MAX_CALLS 도달, 수집 중단")
                return pd.DataFrame(raw_rows), pd.DataFrame(raw_rows)

            url = f"{base_url}?query={enc_kw}&display={page_num}&start={start}&sort={sort}"

            request = urllib.request.Request(url)
            request.add_header("X-Naver-Client-Id", client_id)
            request.add_header("X-Naver-Client-Secret", client_secret)

            try:
                response = urllib.request.urlopen(request)
                api_calls += 1

                if response.getcode() != 200:
                    print(f"[ERROR] {kw} / start={start} / code={response.getcode()}")
                    break

                data = json.loads(response.read().decode("utf-8"))
                items = data.get("items", [])
                if not items:
                    break

                oldest_date_in_page = None

                for item in items:
                    pub_dt = parse_pubdate(item["pubDate"])
                    pub_date_str = pub_dt.strftime("%Y-%m-%d")

                    if oldest_date_in_page is None or pub_date_str < oldest_date_in_page:
                        oldest_date_in_page = pub_date_str

                    if pub_date_str >= target_date_str:
                        raw_rows.append({
                            "query_keyword": kw,
                            "pub_datetime": pub_dt,
                            "pub_date": pub_date_str,
                            "title_raw": item.get("title", ""),
                            "description_raw": item.get("description", ""),
                            "link": item.get("link", ""),
                            "originallink": item.get("originallink", ""),
                            "api_start": start,
                            "collected_at": datetime.now()
                        })

                # break: 페이지의 가장 오래된 날짜가 target_date보다 과거면 종료
                if oldest_date_in_page and oldest_date_in_page < target_date_str:
                    break

                if sleep_sec:
                    time.sleep(sleep_sec)

            except Exception as e:
                print(f"[EXCEPTION] {kw} / start={start} / {e}")
                break

    raw_df = pd.DataFrame(raw_rows)

    if raw_df.empty:
        clean_df = raw_df.copy()
        print(f"API calls used: {api_calls}")
        return raw_df, clean_df

    clean_df = raw_df.copy()
    clean_df["title"] = clean_df["title_raw"].apply(clean_html)
    clean_df["description"] = clean_df["description_raw"].apply(clean_html)

    clean_df = clean_df[[
        "query_keyword", "pub_datetime", "pub_date",
        "title", "description", "link", "originallink",
        "api_start", "collected_at"
    ]]

    print(f"API calls used: {api_calls}")
    print(f"raw rows: {len(raw_df)}")

    return raw_df, clean_df


In [5]:
# 1-2. 실행

keywords = ["자금세탁", "배임", "횡령", "탈세"]
target_date ='2026-02-20' # 원하는 날짜로 바꾸기

raw_df, clean_df = get_raw_and_clean_df(keywords, target_date, sleep_sec=0.1)

# display(raw_df.head())
# display(clean_df.head())


API calls used: 30
raw rows: 2907


In [6]:
# clean_df.loc[clean_df['pub_date']=='2026-02-23',:]

### 2. 뉴스 기사 분석

0) 분석 대상
    - 뉴스 제목 (1st)
    - 뉴스 본문

1) 실제 금융범죄 사건인지 분석
    - 형 확정 (1심, 2심, 최종) 여부
    - 인사발령, 제도 도입 등 관련성 낮은 기사 제외

2) 혐의자 분석
    - 고유명사(이름) 추출
    - 고유명사(이름)이 익명이 아닌 기사


In [55]:
# ----------------------------
# 1. 사전 정의
# ----------------------------

COMMON_SURNAMES = {
"김","이","박","최","정","강","조","윤","장","임","오","한","신","서","권","황","안","송","전","홍","유","고","문","양","손","배","백","허","남","심","노","하","곽","성","차","주","우","구","민","류","나","진","지","엄","채","원","천","방","공","현","함","변","염","여","추","도","소","석","선","설","마","길","연","위","표","명","기","반","라","왕","금","옥","육","인","맹","제","모","남","탁","국"
}

PRISON_PAT = r"(징역\s*\d+\s*(년|개월))|(\d+\s*(년|개월)\s*형)"
EXECUTION_WORDS = ["선고","확정","법정구속","실형"]

ANON_PATTERNS = [
    r"(?:^|[^A-Za-z가-힣])([A-Za-z]|[Ａ-Ｚ])\s*씨", # A씨
    r"(?:^|[^가-힣])[가-힣]\s*씨",                  # 김씨
    r"[가-힣]?\s*모\s*씨",                          # 김모씨
    r"[○●□■]{1,4}\s*씨",                        # ㅇㅇ씨
    r"[가-힣][○●□■]{1,4}"
]

POLICY_WORDS = [
    "도입","시행","발표","개정","추진",
    "가이드라인","정책","방안"
]

# ----------------------------
# 2. 유틸 함수
# ----------------------------

def is_personnel_article(title):
    return title.strip().startswith("[인사]")

def has_sentence_execution(title):
    if re.search(PRISON_PAT, title):
      return True
        # if "구형" not in title:
            # return True
    if any(word in title for word in EXECUTION_WORDS):
        return True
    return False

# 실명추출 - 보완필요사항: 외자 이름, 복성이름 포함 로직
# meCab 이용
def extract_name_candidates(title):
    pos_results = mecab.pos(title)
    candidates = []
    for word, pos in pos_results:
            # 오직 고유명사(NNP)만 필터링!
            if pos == 'NNP':
                # 성씨로 시작하고 2~4글자인지 확인
                if 2 <= len(word) <= 4 and word[0] in COMMON_SURNAMES:
                  candidates.append(word)
    return list(set(candidates))

"""
#1. komoran- "김숙진" -> "김숙"으로 잡는 오류
def extract_name_candidates(title):
    # tokens = re.findall(r"[가-힣]{3,4}", title)
    nouns = [w for w, pos in komoran.pos(title) if pos in ["NNP", "NNG"]] # ,"NNG"
    candidates = []
    for noun in nouns:
        if 2<=len(noun)<=4 and noun[0] in COMMON_SURNAMES:
            candidates.append(noun)
    return list(set(candidates))
"""
"""
#2. okt- 형태소 분류 구조가 심플해서 오류없이 명사 잡아냄, 분류체계가 적어서 고유명사만 선별 불가
def extract_name_candidates(title):
    # tokens = re.findall(r"[가-힣]{3,4}", title)
    nouns = okt.nouns(title)
    candidates = []
    for noun in nouns:
        # 성씨 조건 + 길이 조건 (2~4자)
        if 2 <= len(noun) <= 4 and noun[0] in COMMON_SURNAMES:
          candidates.append(noun)
    return list(set(candidates))
"""

def has_anonymous(text):
    return any(re.search(p, text or "") for p in ANON_PATTERNS)

def is_policy_article(text):
    return any(word in (text or "") for word in POLICY_WORDS)

# 글자수 초과로 뉴스제목 생략 여부
def is_truncated(text):
   return (text or "").strip().endswith('...')

# ----------------------------
# 3. 최종 파이프라인
# ----------------------------

def refined_negative_news(df):
    df = df.copy()
    results = []

    for _, row in df.iterrows():
        title = row["title"]
        body = row.get("description","")

        reason_list = []
        truncated_yn = "N"  # 뉴스제목 생략 여부- 제목 생략 없음(기본값)
        final = "확인필요"

        # 0️⃣ 뉴스제목 생략 여부
        if is_truncated(title):
          truncated_yn = "Y"
          # truncated_yn.append("데이터 생략(말줄임표)")

        # 1️⃣ [인사] 즉시 제외
        if is_personnel_article(title):
            final = "N"
            reason_list.append("인사기사")

        else:
            execution_flag = has_sentence_execution(title)

            # 2️⃣ 실형 집행 없으면 즉시 제외
            if not execution_flag:
                final = "N"
                reason_list.append("실형 집행 언급 없음")

            else:
              reason_list.append("실형 선고 기사")
              name_candidates = extract_name_candidates(title)

              # 3️⃣ 실형 + 이름 있으면 Y
              if name_candidates:
                  final = "Y"
                  # reason_list.append("실형 선고 기사")
                  if re.search("무죄", title):
                    reason_list.append("무죄")
                  else:
                    pass
              else:
                  final = "확인필요"
                  reason_list.append("이름 미확인")

        # 4️⃣ 확인필요 그룹만 본문 추가 검증
        if final == "확인필요":

            if has_anonymous(body):
                final = "N"
                reason_list.append("본문에 익명 혐의자 표현")

            elif is_policy_article(body):
                final = "N"
                reason_list.append("제도/정책 기사")

        results.append({
            **row,
            "name_candidates": extract_name_candidates(title),
            "execution_flag": has_sentence_execution(title),
            "제목잘림여부": truncated_yn,
            "확인내용": " / ".join(reason_list),
            "부정적 뉴스 여부": final
        })

    return pd.DataFrame(results)

In [57]:
test1=refined_negative_news(clean_df)
test1.to_csv('test_9_최종_20260228.csv', index=False, encoding='utf-8-sig')

In [9]:
results=refined_negative_news(clean_df)

## STR 보고서 도입부 작성

In [10]:
"""
def make_str_report(date, policy_type, policyholder, insured, relationship, prem_period, premium, product):  #relationship: 관계
  # 개인청약
  if policy_type == "개인":
    # 기본형
    if policyholder and policyholder == insured:
      return f"{date} 본인을 계,피로 {prem_period} {premium}의 {product}를 청약함"
    # 계약자!=피보험자
    elif policyholder and policyholder != insured:
      return f"{date} 본인을 계약자, {relationship} {insured}를 피보험자로하여 {prem_period} {premium}의 {product}를 청약함"
    else:
      pass
  # 법인청약
  elif policy_type == "법인":
      return f"{date} {policyholder}를 계약자, {relationship} {insured}를 피보험자로하여 {prem_period} {premium}의 {product}를 청약함"  #relationship: 대표/직원
  else:
    pass
"""

'\ndef make_str_report(date, policy_type, policyholder, insured, relationship, prem_period, premium, product):  #relationship: 관계\n  # 개인청약\n  if policy_type == "개인":\n    # 기본형\n    if policyholder and policyholder == insured:\n      return f"{date} 본인을 계,피로 {prem_period} {premium}의 {product}를 청약함"\n    # 계약자!=피보험자\n    elif policyholder and policyholder != insured:\n      return f"{date} 본인을 계약자, {relationship} {insured}를 피보험자로하여 {prem_period} {premium}의 {product}를 청약함"\n    else:\n      pass\n  # 법인청약\n  elif policy_type == "법인":\n      return f"{date} {policyholder}를 계약자, {relationship} {insured}를 피보험자로하여 {prem_period} {premium}의 {product}를 청약함"  #relationship: 대표/직원\n  else:\n    pass\n'

In [11]:
"""
date= '2025-06-05'
policy_type= '개인'
policyholder= '이정자'
insured= '이정자'
relationship= ''
prem_period= '일시납'
premium= '21,300 달러'
product= '외화연금보험'

make_str_report(date, policy_type, policyholder, insured, relationship, prem_period, premium, product)
"""

"\ndate= '2025-06-05'\npolicy_type= '개인'\npolicyholder= '이정자'\ninsured= '이정자'\nrelationship= ''\nprem_period= '일시납'\npremium= '21,300 달러'\nproduct= '외화연금보험'\n\nmake_str_report(date, policy_type, policyholder, insured, relationship, prem_period, premium, product)\n"